In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
matplotlib.rcParams.update({
        'font.size': 16,
        'legend.fontsize': 10,
        'lines.linewidth': 2,
    })

In [ ]:
pickle_paths = [
    ("B205", "/media/thomas/Extreme SSD/orca_paper_data_files/phase_noise/b205/20240222_203345_stacking_stats_20240228_161251_bin187.pickle"),
    ("X310", "/media/thomas/Extreme SSD/orca_paper_data_files/phase_noise/x310/20240222_143225-phase-noise-stats.pickle")
]

phase_stats = []

for name, path in pickle_paths:
    with open(path, "rb") as f:
        s = pickle.load(f)
        for k in list(s.keys()):
            s["stack_"+k] = s[k]
        phase_stats.append((name, s))


### Comparison plot

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), sharex=True)
ax_pwr, ax_ph = axs

for (title, data) in phase_stats:
    pwr_normalized = data["stack_signal_peak_pwr_mean"] / data["stack_signal_peak_pwr_mean"][0]
    pwr_db_normalized = 10*np.log10(pwr_normalized)

    std_upper = 10*np.log10((data["stack_signal_peak_pwr_mean"] + np.sqrt(data["stack_signal_peak_pwr_variance"]))/data["stack_signal_peak_pwr_mean"][0])
    std_lower = 10*np.log10((data["stack_signal_peak_pwr_mean"] - np.sqrt(data["stack_signal_peak_pwr_variance"]))/data["stack_signal_peak_pwr_mean"][0])
    
    l = ax_pwr.plot(data["actual_stack_t"], pwr_db_normalized, label=title+" Mean")
    c = l[0].get_color()
    ax_pwr.plot(data["actual_stack_t"], std_upper, linestyle="--", color=c, linewidth=1, label="+/- 1 Std Dev")
    ax_pwr.plot(data["actual_stack_t"], std_lower, linestyle="--", color=c, linewidth=1)

    #ax_pwr.fill_between(data["actual_stack_t"], std_lower, std_upper, alpha=0.5)

    ax_pwr.set_ylabel("Signal Power [dB]\n(relative to single pulse)")
    ax_pwr.semilogx()
    ax_pwr.legend()

    ph = np.degrees(data["stack_signal_peak_phase"])
    ph_var = np.degrees(data["stack_signal_peak_phase_variance"])

    # ax_ph.plot(data["actual_stack_t"], ph)
    # # Add a shaded region for the variance
    # ax_ph.fill_between(data["actual_stack_t"], ph - np.sqrt(ph_var),
    #                     ph + np.sqrt(ph_var), alpha=0.4)

    ax_ph.plot(data["actual_stack_t"], (data["stack_signal_peak_phase_variance"]), label=title)
    ax_ph.set_ylabel("Variance of Signal Phase [rad$^2$]")
    ax_ph.loglog()
    ax_ph.set_xlabel("Wall Clock Integration Time [s]")
    ax_ph.legend(loc='lower left')

for ax in axs:
    ax.grid(True)

fig.tight_layout()
fig.savefig("phase_noise_summary.png", dpi=300)
plt.show()

### Single file plot

In [ ]:
stats = phase_stats[1]
data = stats[1]
title = stats[0]
print(title)

fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True)
ax_pwr, ax_ph = axs

ax_pwr.plot(data["actual_stack_t"], data["stack_signal_peak_pwr_mean"])
# Add a shaded region for the variance
ax_pwr.fill_between(data["actual_stack_t"], data["stack_signal_peak_pwr_mean"] - np.sqrt(data["stack_signal_peak_pwr_variance"]),
                    data["stack_signal_peak_pwr_mean"] + np.sqrt(data["stack_signal_peak_pwr_variance"]), alpha=0.4)
ax_pwr.set_title("Signal Power")
ax_pwr.loglog()

ph = np.degrees(data["stack_signal_peak_phase"])
ph_var = np.degrees(data["stack_signal_peak_phase_variance"])

# ax_ph.plot(data["actual_stack_t"], ph)
# # Add a shaded region for the variance
# ax_ph.fill_between(data["actual_stack_t"], ph - np.sqrt(ph_var),
#                     ph + np.sqrt(ph_var), alpha=0.4)

ax_ph.plot(data["actual_stack_t"], (data["stack_signal_peak_phase_variance"]))
ax_ph.set_title("Variance of Signal Phase")
ax_ph.loglog()
ax_ph.set_xlabel("Wall Clock Integration Time [s]")

for ax in axs:
    ax.grid(True)

plt.show()